# Third prompt

長文は要約して出力した結果に対し、3つ目のプロンプトを追加し、コンテキストを元に文を作成できているかを判定する。

In [1]:
import os
import pandas as pd
import openai
import datetime
import tiktoken
from sudachipy import tokenizer
from sudachipy import dictionary
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from pydantic import BaseModel, Field

In [2]:
# OpenAI APIキーを設定
openai.api_key = input()

In [3]:
# OpenAIの言語モデルを設定（ここではGPT-3を使用）
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)

C:\Users\tatsu\AppData\Local\Temp\ipykernel_38828\1591884372.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)


## 回答ファイルを読み込む

In [4]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("../submit/20241005-0121_predictions.csv", encoding="utf-8")

In [1]:
query_df.head()

In [7]:
query_df = query_df.rename(columns={"answer": "tmp_answer"})

In [8]:
query_df.columns

Index(['index', 'problem', 'full_answer', 'tmp_answer', 'evidence'], dtype='object')

In [20]:
# LLMを使って文章の修正を行う関数
def check_answer(problem: str, tmp_answer: str, evidence: str) -> str:

    check_prompt = PromptTemplate(
        input_variables=["problem", "full_answer", "evidence"],
        template=
            """あなたは回答の確からしさを判定する優れたチェックシステムです。
            問題に対し、仮の回答とその根拠が与えられています。
            仮の回答が根拠から導き出せていない場合、「分かりません。」と回答してください。
            それ以外の場合、仮の回答を1文字も変えずそのまま回答してください。

                f"問題: {problem}\n\n"
                f"仮の回答: {tmp_answer}\n"
                f"根拠: {evidence}\n"
            回答:"""
    )
    chain = check_prompt | llm

    response = chain.invoke(
        {"problem": problem, "tmp_answer": tmp_answer, "evidence": evidence}
    )
    return response.content

In [21]:
def check_and_summarize_answers(query_df: pd.DataFrame) -> pd.DataFrame:
    # "answer" 列に対して処理を適用
    query_df["answer"] = query_df["tmp_answer"]
    for i in range(len(query_df.index)):
        print(i)
        query_df["answer"][i] = check_answer(query_df["problem"][i], query_df["tmp_answer"][i], query_df["evidence"][i])
        print(query_df["answer"][i])
    return query_df

In [2]:
query_df = check_and_summarize_answers(query_df)

In [3]:
query_df.head(20)

In [44]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "../submit/predictions.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)

In [45]:
# backup
dt_now = datetime.datetime.now()
ymdm = dt_now.strftime("%Y%m%d-%H%M")

query_df[['index', 'problem', 'full_answer', 'answer', 'evidence']].to_csv(
    f"../submit/{ymdm}_predictions.csv",
    index=False,
    header=True,
    encoding="utf-8-sig"
)